# environment setting

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
import seaborn as sns
from matplotlib import pyplot as plt
import math
from itertools import product
import os

print(os.getcwd())

In [ ]:
SCOREDATA_PATH = '/Users/wenjunzhu/Desktop/dataAnalysis/questionnaire/datafile/data.csv'
PICTURE_PATH = '/Users/wenjunzhu/Desktop/dataAnalysis/questionnaire/pictures'

# correlation analysis

In [ ]:
# correlation analysis
df = pd.read_csv(SCOREDATA_PATH, skipinitialspace=True)
# print(df.dtypes)
corr_matrix = df.iloc[:,1:].corr(method="pearson")
print(corr_matrix)

# p values calculation
p_values = np.zeros_like(corr_matrix)
for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[1]):
        _, p_values[i, j] = stats.pearsonr(df.iloc[:, i+1], df.iloc[:, j+1])

# multiple comparison correction
p_values_flat = p_values[np.triu_indices_from(p_values, k=1)]
_, p_adjusted, _, _ = multipletests(p_values_flat, method='fdr_bh')
p_adjusted_matrix = np.zeros_like(p_values)
p_adjusted_matrix[np.triu_indices_from(p_adjusted_matrix, k=1)] = p_adjusted
p_adjusted_matrix += p_adjusted_matrix.T

# extract adjusted p matrix without column_id
df_p = pd.DataFrame(p_adjusted_matrix, columns=df.columns[1:], index = df.columns[1:])
print(df_p) 


# heatmap configuration
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
fig, ax = plt.subplots(figsize=(10, 6)) # ???

sns.heatmap(
    corr_matrix,
    annot = True,
    fmt = ".2f",
    linewidth = .5,
    vmax = 1.00,
    vmin = -1.00,
    cmap = "coolwarm", # RdBu_r  bwr
    mask = mask,
    )

# frame configuration  # ???
ax.axhline(y=0, color='k',linewidth=3)
ax.axhline(y=corr_matrix.shape[1], color='k',linewidth=3)
ax.axvline(x=0, color='k',linewidth=3)
ax.axvline(x=corr_matrix.shape[0], color='k',linewidth=3)

# add asterisks where the p values are less than .05  # ???
for i, j in product(range(corr_matrix.shape[0]), range(corr_matrix.shape[1])):
    if df_p.iloc[i, j] < 0.05 and df_p.iloc[i, j] > 0.01:
        ax.text(
            j + 0.5,
            i + 1.0,
            "*",
            ha="center",
            va="bottom",
            color="white",
            fontsize=12,
        )
    elif df_p.iloc[i, j] < 0.01:
        ax.text(
            j + 0.5,
            i + 1.0,
            "**",
            ha="center",
            va="bottom",
            color="white",
            fontsize=12,
        )

plt.title("Correlation Matrix")
plt.show()

# regression analysis

In [ ]:
# regression analysis
# dependent var: SSCS, independent var: notWorry(-.32), self-regul(.3), trusting(.33), accuracy(.4)

y = df["SSCS"]
x = df[["MAIA_notWorry", "MAIA_self-regul", "MAIA_trusting", "accuracy"]]

# standradization
scaler = StandardScaler()
y_standardized = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

x_standardized = scaler.fit_transform(x)
x_standardized_df = pd.DataFrame(x_standardized, columns=x.columns)

x_standardized_df = sm.add_constant(x_standardized_df) # add intercept column

# conduct regression
model = sm.OLS(y_standardized, x_standardized_df).fit()

print(model.summary())

In [ ]:
# regression analysis
# dependent var: LMS-engagement, independent var: notDistract(.33), emotion(-.41)

y = df["LMS_E"]
x = df[["MAIA_notDistract", "MAIA_emotion"]]

# standradization
scaler = StandardScaler()
y_standardized = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

x_standardized = scaler.fit_transform(x)
x_standardized_df = pd.DataFrame(x_standardized, columns=x.columns)
x_standardized_df = sm.add_constant(x_standardized_df)

model = sm.OLS(y_standardized, x_standardized_df).fit()

print(model.summary())

In [ ]:
# regression analysis
# dependent var: DAT, independent var: attention(-.38), bodylisten(.48), trusting(-.31), accuracy (.31)

y = df["DAT_score"]
x = df[["MAIA_attention", "MAIA_bodyListen", "MAIA_trusting", "accuracy"]]

# standradization
scaler = StandardScaler()
y_standardized = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

x_standardized = scaler.fit_transform(x)
x_standardized_df = pd.DataFrame(x_standardized, columns=x.columns)
x_standardized_df = sm.add_constant(x_standardized_df)

model = sm.OLS(y_standardized, x_standardized_df).fit()

print(model.summary())

# main figures

## SSCS

In [ ]:
# plot graph of SSCS and correlated factors

# extract the labels of pairs that |r| > .4 and figure configration

y = "SSCS"
x = ["MAIA_notWorry", "MAIA_self-regul", "MAIA_trusting", "accuracy"]

# プロット数に応じた行と列の数を計算
n_pairs = len(x)
n_cols = 2  # 一列に2つのプロットを配置
n_rows = math.ceil(n_pairs / n_cols)  # 行数を計算

# 大きなプロット領域を作成
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()  # 軸を1次元リストに変換

# プロットを描画
for idx, label in enumerate(x):
    if idx == 0:
        sns.regplot(data=df, x=label, y=y, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "blue"})
    else:
        sns.regplot(data=df, x=label, y=y, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "red"})
    axes[idx].set_title(f'{label} & {y}')
    axes[idx].set_xlabel(label)
    axes[idx].set_ylabel(y)

# 残りの空プロットを非表示
for idx in range(len(x), len(axes)):
    axes[idx].axis('off')

# 全体のレイアウトを調整
plt.tight_layout()
plt.show()


## LMS

In [ ]:
# plot graph of LMS_NS & MAIA_bodyListen

x = "MAIA_bodyListen"
y = "LMS_NS"
title = f"{x} & {y}"

slope, intercept, r_value, p_value, std_err = linregress(df[x], df[y])

# Seabornのjointplotを作成
fig = sns.regplot(
    data=df,
    x=x,
    y=y,        
    scatter_kws={'color':'black', 'alpha': 0.6}, 
    line_kws={"color": "blue"}
)
fig.set_title(title)

plt.show()

In [ ]:
# plot graph of LMS_E and correlated factors


y = "LMS_E"
x = ["MAIA_notDistract", "MAIA_emotion"]

# プロット数に応じた行と列の数を計算
n_pairs = len(x)
n_cols = 2  # 一列に2つのプロットを配置
n_rows = math.ceil(n_pairs / n_cols)  # 行数を計算

# 大きなプロット領域を作成
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()  # 軸を1次元リストに変換

# プロットを描画
for idx, label in enumerate(x):
    if idx == 1:
        sns.regplot(data=df, x=label, y=y, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "blue"})
    else:
        sns.regplot(data=df, x=label, y=y, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "red"})
    axes[idx].set_title(f'{label} & {y}')
    axes[idx].set_xlabel(label)
    axes[idx].set_ylabel(y)

# 残りの空プロットを非表示
for idx in range(len(x), len(axes)):
    axes[idx].axis('off')

# 全体のレイアウトを調整
plt.tight_layout()
plt.show()


## DAT

In [ ]:
# plot graph of DAT and correlated factors

y = "DAT_score"
x = ["MAIA_attention", "MAIA_bodyListen", "MAIA_trusting", "accuracy"]

# プロット数に応じた行と列の数を計算
n_pairs = len(x)
n_cols = 2  # 一列に2つのプロットを配置
n_rows = math.ceil(n_pairs / n_cols)  # 行数を計算

# 大きなプロット領域を作成
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()  # 軸を1次元リストに変換

# プロットを描画
for idx, label in enumerate(x):
    if idx == 0 or idx == 2:
        sns.regplot(data=df, x=label, y=y, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "blue"})
    else:
        sns.regplot(data=df, x=label, y=y, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "red"})
    axes[idx].set_title(f'{label} & {y}')
    axes[idx].set_xlabel(label)
    axes[idx].set_ylabel(y)

# 残りの空プロットを非表示
for idx in range(len(x), len(axes)):
    axes[idx].axis('off')

# 全体のレイアウトを調整
plt.tight_layout()
plt.show()


# other figures

In [ ]:
# extract the labels of pairs that |r| > .4 and figure configration
high_cor_pairs = []

corr_intreset = corr_matrix.loc["SSCS":"DAT_score","MAIA_noticing":"accuracy"]
print(corr_intreset)

for i in range(corr_intreset.shape[0]):
    for j in range(corr_intreset.shape[1]):  
        r = corr_intreset.iloc[i, j]
        print(r)
        if abs(r) >= .35:  
            high_cor_pairs.append([corr_intreset.columns[j], corr_intreset.index[i]])

print(high_cor_pairs)

# プロット数に応じた行と列の数を計算
n_pairs = len(high_cor_pairs)
n_cols = 2  # 一列に2つのプロットを配置
n_rows = math.ceil(n_pairs / n_cols)  # 行数を計算

# 大きなプロット領域を作成
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axes = axes.flatten()  # 軸を1次元リストに変換

# プロットを描画
for idx, (x_col, y_col) in enumerate(high_cor_pairs):
    if idx == 0 or idx == 3:
        sns.regplot(data=df, x=x_col, y=y_col, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "red"})
    else:
        sns.regplot(data=df, x=x_col, y=y_col, ax=axes[idx], 
                    scatter_kws={"color": "black", "alpha":0.6}, line_kws={"color": "blue"})
    axes[idx].set_title(f'{x_col} & {y_col}')
    axes[idx].set_xlabel(x_col)
    axes[idx].set_ylabel(y_col)

# 残りの空プロットを非表示
for idx in range(len(high_cor_pairs), len(axes)):
    axes[idx].axis('off')

# 全体のレイアウトを調整
plt.tight_layout()
plt.show()


In [ ]:
# plot graph initial verson
# "accuracy & DAT"

x = "accuracy"
y = "DAT_score"
title = f"{x} & {y}"

# regression line
slope, intercept, r_value, p_value, std_err = linregress(df[x], df[y])
x_fit = np.linspace(df[x].min(), df[x].max(), 5)
y_fit = slope * x_fit + intercept
plt.plot(x_fit, y_fit, 
         color="red", 
         linestyle='-', 
         label=f"y = {slope:.2f}x + {intercept:.2f}")

# plot
plt.scatter(
    df[x], df[y],
    color = "#000000"
            )
plt.title(f"{title} (r = {round(corr_matrix.loc[x,y], 2)})")
plt.xlabel(x)
plt.ylabel(y)
plt.xlim(0, 100)
plt.ylim(70, 100)
plt.legend()

In [ ]:
# accuracy & DAT_score definitive verson

x = "accuracy"
y = "DAT_score"
title = f"{x} & {y}"

slope, intercept, r_value, p_value, std_err = linregress(df[x], df[y])

# Seabornのjointplotを作成
fig = sns.jointplot(
    data=df,
    x=x,
    y=y,
    kind="reg",
    color="#fa8072",              
    scatter_kws={'color':'black', 'alpha': 0.6}, # 散布図の透明度
    xlim=(0,100),
    ylim=(75,100)
)

# タイトルの追加（matplotlibを使用）
# fig.fig.suptitle(f"{title} (r = {r_value:.2f})", y=1.02)

plt.show()

In [ ]:
# plot graph
# MAIA & DAT

title = "MAIA & DAT"
x = list(df.columns[4:5]) + list(df.columns[7:9])
print(x)
y = "DAT_score"

# convert wide df to long df
df_long = pd.melt(
    df, 
    id_vars=y,
    value_vars=x,
    var_name="MAIA_subscale",
    value_name="MAIA_score"
    )

# figure configuration
fig = sns.jointplot(
    data=df_long,
    x="MAIA_score",
    y=y,
    xlim=(1,5),
    ylim=(70,100),
    palette="rainbow", # cool or rainbow is preferred
    hue="MAIA_subscale",
    )